In [1]:
import pandas as pd

In [2]:
houses=pd.read_csv("..//data//house_data.csv")

In [12]:
def label_counts(data):
    label_count = {} 
    for row in data:
        label = row[0]
        if label not in label_count:
            label_count[label] = 0
        label_count[label] += 1
    return label_count

In [13]:
def gini(data):
    label_count = label_counts(data)
    impurity = 1
    for label in label_count:
        label_prob = label_count[label] / float(len(data))
        impurity -= label_prob**2
    return impurity

In [85]:
def create_split(data,col,val):
    right = data[data[col]>val]
    left = data[data[col]<=val]
    return left,right

In [6]:
def information_gain(left, right, impurity):
    p = float(len(left)) / (len(left) + len(right))
    gain = impurity - p * gini(left.values) - (1 - p) * gini(right.values)
    return gain

In [86]:
def build_tree(data, label_col, tree_dict,depth,max_depth ):
    if depth==max_depth:
        end_dict={}
        end_dict["label_count"]=label_counts(data.values)
        end_dict['left']={}
        end_dict['right']={}
        return end_dict

    max_info_gain=-9999
    sel_col=None
    sel_val=None
    initial_impurity = gini(data.values)
    for col in data:
        if col != label_col:
            for val in data[col].unique().tolist():
                info_gain=0
                left , right = create_split(data,col,val)
                info_gain = information_gain(left,right,initial_impurity)

                if info_gain >max_info_gain:
                    max_info_gain=info_gain
                    sel_col = col
                    sel_val = val

    if sel_col==None:
        end_dict={}
        end_dict["label_count"]=label_counts(data.values)
        end_dict['left']={}
        end_dict['right']={}
        return end_dict

    tree_dict["col"]=sel_col
    tree_dict["val"]=sel_val
    tree_dict["label_count"]=label_counts(data.values)
    tree_dict["info_gain"]=max_info_gain
    tree_dict["impurity"]=initial_impurity
    left , right = create_split(data,sel_col,sel_val)
    tree_dict["left"]={}
    tree_dict["left"]=build_tree(left,label_col,tree_dict["left"],depth+1,max_depth)
    tree_dict["right"]={}
    tree_dict["right"]=build_tree(right,label_col,tree_dict["right"],depth+1,max_depth)

    return tree_dict

In [87]:
decision_tree=build_tree(houses,"in_sf",{},1,4)

In [124]:
def predict(decision_tree,data):
    traverse = True
    while traverse:
        col = decision_tree['col']
        val = decision_tree['val']
        if data[col]>val:
            decision_tree = decision_tree['right']
        else :
            decision_tree = decision_tree['left']
        if not decision_tree['right'] and not decision_tree['left']:
            traverse = False
    if 1.0 in decision_tree['label_count'] and 0.0 in decision_tree['label_count']:
        label_flag = 1 if decision_tree['label_count'][1.0]> decision_tree['label_count'][0.0] else 0
        return label_flag
    elif 1.0 in decision_tree['label_count']:
        return 1
    else:
        return 0
        

In [125]:
houses['predicted_label']=houses.apply(lambda row : predict(decision_tree,row),axis =1)

In [126]:
prediction = houses['in_sf']==houses['predicted_label']

In [127]:
prediction.mean()

0.89634146341463417